In [2]:
import csv
import os
import requests
from bs4 import BeautifulSoup
import openpyxl
import re
import pandas as pd

# Function to read stop words from a file
def read_file_words(file_path, encodings=['utf-8', 'latin-1', 'ISO-8859-1', 'windows-1252']):
    for encoding in encodings:
        try:
            with open(file_path, 'r', encoding=encoding) as file:
                stop_words = [line.strip() for line in file]
            return stop_words
        except UnicodeDecodeError:
            continue
    raise UnicodeDecodeError("Unable to decode the file using any of the specified encodings.")


# Function to calculate positive score
def calculate_positive_score(text):
    positive_score = sum(1 for word in text.split() if word.lower() in positive_words)
    return positive_score

# Function to calculate negative score
def calculate_negative_score(text):
    negative_score = sum(1 for word in text.split() if word.lower() in negative_words)
    return negative_score
def calculate_total_score(positive_score, negative_score):
    if(positive_score+negative_score == 0):
        return int(1)
    return positive_score + negative_score
# Function to calculate polarity score
def calculate_polarity_score(positive_score, negative_score):
    return positive_score - negative_score

# Function to calculate subjective score
def calculate_subjective_score(positive_score, negative_score):
    return positive_score + negative_score

# Function to split text into sentences
def split_into_sentences(text):
    # Define regular expression pattern for sentence boundary
    pattern = r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s'
    # Split text into sentences using regular expression
    sentences = re.split(pattern, text)
    return sentences
# Function to calculate average sentence length
def calculate_average_sentence_length(text):
    # Split text into sentences
    sentences = split_into_sentences(text)
    # Calculate total number of words and total number of sentences
    total_words = sum(len(sentence.split()) for sentence in sentences)
    total_sentences = len(sentences)
    # Calculate average sentence length
    if total_sentences > 0:
        average_length = total_words / total_sentences
    else:
        average_length = 0
    return average_length

# Function to count syllables in a word
def count_syllables(word):
    # Simple syllable counting algorithm (not perfect)
    # Count consecutive vowels as syllables, but exclude certain combinations
    vowels = 'aeiouy'
    count = 0
    word = word.lower()
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
    if word.endswith('e'):
        count -= 1
    if count == 0:
        count = 1  # At least one syllable for words like 'a', 'I'
    return count

# Function to calculate percentage of complex words in text
def calculate_complexity_percentage(text, threshold_syllables=3):
    # Tokenize text into words
    words = re.findall(r'\b\w+\b', text.lower())
    # Count total words and complex words
    total_words = len(words)
    complex_words = sum(1 for word in words if count_syllables(word) >= threshold_syllables)
    # Calculate percentage
    if total_words > 0:
        percentage = (complex_words / total_words) * 100
    else:
        percentage = 0
    return percentage

# Function to count syllables in a word
def count_syllables(word):
    # Simple syllable counting algorithm (not perfect)
    # Count consecutive vowels as syllables, but exclude certain combinations
    vowels = 'aeiouy'
    count = 0
    word = word.lower()
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
    if word.endswith('e'):
        count -= 1
    if count == 0:
        count = 1  # At least one syllable for words like 'a', 'I'
    return count

# Function to calculate percentage of complex words in text
def calculate_complexity_percentage(text, threshold_syllables=3):
    # Tokenize text into words
    words = re.findall(r'\b\w+\b', text.lower())
    # Count total words and complex words
    total_words = len(words)
    complex_words = sum(1 for word in words if count_syllables(word) >= threshold_syllables)
    # Calculate percentage
    if total_words > 0:
        percentage = (complex_words / total_words) * 100
    else:
        percentage = 0
    return percentage
# Function to calculate Gunning Fog Index
def calculate_gunning_fog_index(text):
    # Tokenize text into sentences
    sentences = re.split(r'[.!?]', text)
    # Count total words and complex words
    total_words = 0
    complex_words = 0
    for sentence in sentences:
        # Tokenize sentence into words
        words = re.findall(r'\b\w+\b', sentence.lower())
        # Count total words
        total_words += len(words)
        if(total_words == 0):
            total_words = 1
        # Count complex words
        for word in words:
            if count_syllables(word) >= 3:
                complex_words += 1
    # Calculate average sentence length (ASL)
    average_sentence_length = total_words / len(sentences)
    # Calculate percentage of complex words
    percentage_complex_words = (complex_words / total_words) * 100
    # Calculate Gunning Fog Index
    fog_index = 0.4 * (average_sentence_length + percentage_complex_words)
    return fog_index



# Function to calculate average word per sentence
def calculate_avg_words_per_sentence(text):
    # Tokenize text into sentences
    sentences = re.split(r'[.!?]', text)
    # Count total words and total sentences
    total_words = 0
    total_sentences = len(sentences)
    # Iterate through each sentence
    for sentence in sentences:
        # Tokenize sentence into words
        words = re.findall(r'\b\w+\b', sentence)
        # Update total words count
        total_words += len(words)
    # Calculate average words per sentence
    if total_sentences > 0:
        avg_words_per_sentence = total_words / total_sentences
    else:
        avg_words_per_sentence = 0
    return avg_words_per_sentence
# Function to count syllables in a word
def count_syllables(word):
    # Simple syllable counting algorithm (not perfect)
    # Count consecutive vowels as syllables, but exclude certain combinations
    vowels = 'aeiouy'
    count = 0
    word = word.lower()
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
    if word.endswith('e'):
        count -= 1
    if count == 0:
        count = 1  # At least one syllable for words like 'a', 'I'
    return count

# Function to calculate counts and averages
def calculate_counts_and_averages(text):
    # Tokenize text into words
    words = re.findall(r'\b\w+\b', text.lower())
    # Initialize counts
    complex_word_count = 0
    total_word_count = len(words)
    total_syllable_count = 0
    # Calculate counts and total syllable count
    for word in words:
        syllable_count = count_syllables(word)
        total_syllable_count += syllable_count
        if syllable_count >= 3:
            complex_word_count += 1
    # Calculate average syllables per word
    if total_word_count > 0:
        avg_syllables_per_word = total_syllable_count / total_word_count
    else:
        avg_syllables_per_word = 0
    return complex_word_count, total_word_count, avg_syllables_per_word

# Function to calculate counts and averages
def calculate_counts_and_averages(text):
    # Tokenize text into words
    words = re.findall(r'\b\w+\b', text.lower())
    # Initialize counts
    personal_pronoun_count = 0
    total_word_count = len(words)
    total_character_count = sum(len(word) for word in words)
    # List of personal pronouns
    personal_pronouns = ["i", "you", "he", "she", "it", "we", "they", "me", "him", "her", "us", "them", "my", "your", "his", "her", "its", "our", "their", "mine", "yours", "hers", "ours", "theirs"]
    # Count occurrences of personal pronouns
    for word in words:
        if word in personal_pronouns:
            personal_pronoun_count += 1
    # Calculate average word length
    if total_word_count > 0:
        avg_word_length = total_character_count / total_word_count
    else:
        avg_word_length = 0
    return personal_pronoun_count, total_word_count, avg_word_length

def remove_duplicates(csv_file, column_index):
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file)
    
    # Remove duplicate rows based on the specified column
    df.drop_duplicates(subset=df.columns[column_index], inplace=True)
    
    # Write the DataFrame back to the same CSV file, overwriting the existing content
    df.to_csv(csv_file, index=False)


# Load the workbook
workbook = openpyxl.load_workbook('Input.xlsx')
# active the workbook
worksheet = workbook.active
# access the input data from xlsx
url_value = worksheet['B1'].value

# Select the 'url' column
url_index = 2
# create a list to store the url in one list
url_values = []

# Read stop words from each txt file and combine them
all_stop_words = []
stop_word_files = read_file_words("stop_words_english.txt")

# If your stop word or any txt file is inside any folder use following for loop code
# for file_name in stop_word_files:
#     file_path = os.path.join("/file_path", file_name)
#     all_stop_words.extend(read_file_words(file_path))

# Read positive and Negative words from txt file
positive_words = read_file_words("positive-words.txt")
negative_words = read_file_words("negative-words.txt")



max_row = worksheet.max_row
# create a loop to store all the url column value in the list
for row in worksheet.iter_rows(min_row=2, max_row=max_row, min_col=url_index, max_col=url_index):
    for cell in row:
        url_values.append(cell.value)

# create a list to store the words
words_sorted = []

# Scrape data from each URL
for url_link in url_values:
    url = url_link
    response = requests.get(url, allow_redirects=False)
    if response.status_code == 200:
        print("Success! Received valid response.")
        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find all paragraph from class tags and extract text
        paragraphs = soup.find_all(class_ = 'mw-body-content')
        
        # for element in paragraphs:
        #     paragraphs = element.find_all('p','ol')
        #     for paragraph in paragraphs:
        #         # Skip specific paragraph(s) from class
        #         if 'paragraph_to_skip' in paragraph.get('wp-block-preformatted', []):
        #             continue
        #         words_sorted.append(paragraph.get_text())

        for element in paragraphs:
            # Check if the element is a div
            if element.name == 'div':
                # Find all paragraphs and ignore <pre> tags within the div
                paragraphs_within_div = element.find_all(['p', 'ol', 'li'])
                for sub_element in paragraphs_within_div:
                    # Check if the sub-element is a <pre> tag
                    if sub_element.name == 'pre':
                        continue  # Skip <pre> tags
                    else:
                        # Append text from non-<pre> sub-elements to words_sorted
                        words_sorted.append(sub_element.get_text())
            else:
                # Append text from non-div elements directly to words_sorted
                words_sorted.append(element.get_text())


                    # words_sorted.append(element.get_text())
        # Extract text from each paragraph and filter out stop words
        for paragraph in words_sorted:  
            words = paragraph.split()
            # Filter out stop words
            filtered_words = [word for word in words if word.lower() not in all_stop_words]
            filtered_text = ' '.join(filtered_words)
            # print(filtered_text)
        positive_score = calculate_positive_score(filtered_text)
        negative_score = calculate_negative_score(filtered_text)
        total_score = calculate_total_score(positive_score,negative_score)
        polarity_score = calculate_polarity_score(positive_score, negative_score)/total_score
        subjective_score = calculate_subjective_score(positive_score, negative_score)/total_score
        average_length = calculate_average_sentence_length(filtered_text)
        complexity_percentage = calculate_complexity_percentage(filtered_text)
        personal_pronoun_count, total_word_count, avg_word_length = calculate_counts_and_averages(filtered_text)
        complex_word_count, total_word_count, avg_syllables_per_word = calculate_counts_and_averages(filtered_text)
        
        gunning_fog_index = calculate_gunning_fog_index(filtered_text)
       
        avg_words_per_sentence = calculate_avg_words_per_sentence(filtered_text)
        
#         You can also print all the output here
#         print("Positive score:", positive_score)
#         print("Negative score:", negative_score)
#         print("Polarity score:", polarity_score)
#         print("Subjective score:", subjective_score)
#         print("Average sentence length:", average_length)
#         print("Percentage of complex words:", complexity_percentage)
#         print("Gunning Fog Index:", gunning_fog_index)
#         print("Average words per sentence:", avg_words_per_sentence)
#         print("Complex word count:", complex_word_count)
        
        
#         print("Total word count:", total_word_count)
#         print("Average syllables per word:", avg_syllables_per_word)

#         print("Personal pronoun count:", personal_pronoun_count)
#         print("Total word count:", total_word_count)
#         print("Average word length:", avg_word_length)
        
        # Write extracted information to a CSV file
        with open('output.csv', mode='a', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            if os.stat('output.csv').st_size == 0:  # Check if file is empty
                writer.writerow(['url', 'POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE', 'SUBJECTIVITY SCORE', 'AVG SENTENCE LENGTH',
                                 'PERCENTAGE OF COMPLEX WORDS', 'FOG INDEX', 'AVG NUMBER OF WORDS PER SENTENCE', 'COMPLEX WORD COUNT', 'WORD COUNT', 'SYLLABLE PER WORD', 'PERSONAL PRONOUNS',
                                 'AVG WORD LENGTH'])
            writer.writerow([url, positive_score, negative_score, polarity_score, subjective_score, average_length,
                             complexity_percentage, gunning_fog_index, avg_words_per_sentence, complex_word_count, total_word_count, avg_syllables_per_word, personal_pronoun_count,
                             avg_word_length])

        
        
        
        
        
        
        
    else:
        print("Error! Failed to receive valid response. Status code:", response.status_code)
csv_file = 'output.csv'  
column_index = 0  
remove_duplicates(csv_file, column_index)
# Don't forget to close the workbook when you're done
workbook.close()


Success! Received valid response.
Success! Received valid response.
Success! Received valid response.
